---
## Sección 1. Configuración inicial

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt

import mlflow
import dagshub
import time

print(f"TensorFlow: {tf.__version__}")
print(f"MLflow: {mlflow.__version__}")

TensorFlow: 2.20.0
MLflow: 3.6.0


In [27]:
# Configurar DagsHub
dagshub.init(repo_owner='404brainnotfound-ai', repo_name='RNA', mlflow=True)

print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Initialized MLflow to track repo "404brainnotfound-ai/RNA"

Repository 404brainnotfound-ai/RNA initialized!

Tracking URI: https://dagshub.com/404brainnotfound-ai/RNA.mlflow


In [28]:
# Verificar disponibilidad de GPU y listar dispositivos físicos
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs disponibles: {len(gpus)}")

if gpus:
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    # Configurar crecimiento de memoria (evita que TensorFlow reserve toda la VRAM)
    tf.config.experimental.set_memory_growth(gpu, True)
    
    # Información adicional sobre la GPU
    print("\nDetalles de GPU:")
    from tensorflow.python.client import device_lib
    local_devices = device_lib.list_local_devices()
    for device in local_devices:
        if device.device_type == 'GPU':
            print(f"  Nombre: {device.physical_device_desc}")
else:
    print("No se detectó ninguna GPU. El entrenamiento usará CPU.")

GPUs disponibles: 1
  GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Detalles de GPU:
  Nombre: device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


I0000 00:00:1763956731.063360   27461 gpu_device.cc:2020] Created device /device:GPU:0 with 2857 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [29]:
# Cargamos lso datos

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Aplanar y normalizar
x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test = x_test.reshape(10000, 784).astype('float32') / 255.0

# Onehot encoding
num_classes = 10
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

In [30]:
def train_and_log(model, run_name, params, epochs=15, batch_size=10):
    """Entrena un modelo y registra en MLflow."""
    
    with mlflow.start_run(run_name=run_name):
        # Registrar parámetros
        mlflow.log_params(params)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        # Entrenar
        start_time = time.time()
        history = model.fit(
            x_train, y_train_cat,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test_cat))
        
        training_time = time.time() - start_time
        
        # Registrar métricas finales
        mlflow.log_metric("training_time", training_time)
        mlflow.log_metric("train_accuracy", history.history['accuracy'][-1])
        mlflow.log_metric("val_accuracy", history.history['val_accuracy'][-1])
        mlflow.log_metric("train_loss", history.history['loss'][-1])
        mlflow.log_metric("val_loss", history.history['val_loss'][-1])
        
        # Guardar modelo
        mlflow.keras.log_model(model, "model")
        
        # Resumen
        print(f"\n{run_name}: Val Acc = {history.history['val_accuracy'][-1]:.4f} | Tiempo = {training_time:.2f}s\n")
        
        return history

---
# Sección 2: Experimentos sin regularización

## Experimento 1: Más neuronas en capa oculta

**Hipótesis**: Si aumentamos de 30 a 128 neuronas mejorará la capacidad de aprendizaje.

In [31]:
# Arquitectura: [784, 128, 10]

model_exp1 = Sequential([
    Dense(128, activation='sigmoid', input_shape=(784,)),
    Dense(10, activation='sigmoid')])

model_exp1.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy'])

params_exp1 = {
    'arquitectura': '[784, 128, 10]',
    'activacion': 'sigmoid',
    'optimizer': 'Adam',
    'learning_rate': 0.001,
    'loss': 'binary_crossentropy'}

history_exp1 = train_and_log(model_exp1, "exp1_mas_neuronas", params_exp1)

/home/enriq/tf-gpu/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1763956835.425238   27461 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2857 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/15


2025-11-23 22:00:41.896691: I external/local_xla/xla/service/service.cc:163] XLA service 0x78b5f8008db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-23 22:00:41.896774: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti, Compute Capability 6.1
2025-11-23 22:00:41.955145: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-23 22:00:42.208536: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600


  43/6000 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.1885 - loss: 0.4918 

I0000 00:00:1763956843.350465   39647 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9048 - loss: 0.0663 - val_accuracy: 0.9443 - val_loss: 0.0367
Epoch 2/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9531 - loss: 0.0318 - val_accuracy: 0.9590 - val_loss: 0.0271
Epoch 3/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - accuracy: 0.9675 - loss: 0.0232 - val_accuracy: 0.9667 - val_loss: 0.0234
Epoch 4/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 33s 5ms/step - accuracy: 0.9753 - loss: 0.0183 - val_accuracy: 0.9720 - val_loss: 0.0197
Epoch 5/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9798 - loss: 0.0152 - val_accuracy: 0.9744 - val_loss: 0.0184
Epoch 6/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9839 - loss: 0.0128 - val_accuracy: 0.9738 - val_loss: 0.0179
Epoch 7/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - accuracy: 0.9862 - loss: 0.0109 - val_accuracy: 0.9755 - val_loss: 0.0173
Epoch 8/15
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.9889 - loss: 0.0094 - val

2025/11/23 22:08:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run exp1_mas_neuronas at: https://dagshub.com/404brainnotfound-ai/RNA.mlflow/#/experiments/0/runs/51719a1a0dc8449fa554b52602339bd6
🧪 View experiment at: https://dagshub.com/404brainnotfound-ai/RNA.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}